In [3]:
import glob
import pandas as pd
catfs = glob.glob("train/train/cat.*.jpg")
catt = [0] * len(catfs)
dogfs = glob.glob("train/train/dog.*.jpg")
dogt = [1] * len(dogfs)
df = pd.DataFrame({
    "path":catfs + dogfs,
    "ans":catt + dogt
})
df

,path,ans
0,train/train\cat.0.jpg,0
1,train/train\cat.1.jpg,0
2,train/train\cat.10.jpg,0
3,train/train\cat.100.jpg,0
4,train/train\cat.1000.jpg,0
...,...,...
24995,train/train\dog.9995.jpg,1
24996,train/train\dog.9996.jpg,1
24997,train/train\dog.9997.jpg,1
24998,train/train\dog.9998.jpg,1


In [5]:
from keras.applications.vgg16 import VGG16
vgg = VGG16(include_top=False, input_shape=(224, 224, 3))
vgg.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [12]:
vgg.layers

In [13]:
from keras.models import Model
from keras.layers import Dense, Dropout, Flatten
# trainable一定要在compile前
for l in vgg.layers:
    l.trainable = False
x = Flatten()(vgg.output)
x = Dense(512, activation="relu")(x)
x = Dropout(0.25)(x)
out = Dense(2, activation="softmax")(x)
model = Model(inputs=vgg.input, outputs=out)
model.summary()

Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0   

In [15]:
model.compile(loss="sparse_categorical_crossentropy",
              optimizer="adam",
              metrics=["accuracy"])

In [21]:
# https://github.com/keras-team/keras-applications/blob/master/keras_applications/imagenet_utils.py
import numpy as np
from keras.preprocessing.image import load_img
from keras.applications.vgg16 import preprocess_input
p = df.iloc[0]["path"]
img = load_img(p, target_size=(224, 224)).convert("RGB")
img_np = np.array(img)
img_pre = preprocess_input(img_np)
img_pre

array([[[ -16.939003 ,   47.221    ,   79.32     ],
        [ -13.939003 ,   50.221    ,   82.32     ],
        [ -10.939003 ,   53.221    ,   85.32     ],
        ...,
        [  15.060997 ,   86.221    ,  121.32     ],
        [  18.060997 ,   85.221    ,  119.32     ],
        [  17.060997 ,   83.221    ,  115.32     ]],

       [[ -16.939003 ,   47.221    ,   79.32     ],
        [ -13.939003 ,   50.221    ,   82.32     ],
        [ -10.939003 ,   53.221    ,   85.32     ],
        ...,
        [  16.060997 ,   88.221    ,  121.32     ],
        [  19.060997 ,   87.221    ,  118.32     ],
        [  18.060997 ,   84.221    ,  116.32     ]],

       [[ -16.939003 ,   47.221    ,   79.32     ],
        [ -13.939003 ,   50.221    ,   82.32     ],
        [ -10.939003 ,   53.221    ,   85.32     ],
        ...,
        [  18.060997 ,   87.221    ,  121.32     ],
        [  20.060997 ,   88.221    ,  119.32     ],
        [  19.060997 ,   85.221    ,  117.32     ]],

       ...,

      